In [7]:
import re
import folium
from typing import List, Tuple, Optional

def map_from_string(text: str,
                    order: str = "lonlat",
                    zoom_start: int = 13,
                    save_path: Optional[str] = None) -> folium.Map:
    """
    Convierte un string con pares numéricos en puntos GPS y los grafica en un mapa.
    
    Parámetros
    ----------
    text : str
        Cadena con pares numéricos; p. ej. "2.12 41.31 | 2.15 41.34 | ..."
        Por defecto se interpreta como (lon, lat) en cada par.
    order : {"lonlat", "latlon"}
        Orden de los pares en el texto.
    zoom_start : int
        Zoom inicial del mapa.
    save_path : str | None
        Si se indica, guarda el HTML del mapa en esa ruta.

    Retorna
    -------
    folium.Map
        Mapa interactivo con los puntos, polilínea y polígono.
    """
    # 1) Capturamos todos los números (soporta +/-, decimales)
    nums = list(map(float, re.findall(r'[-+]?\d+(?:\.\d+)?', text)))
    if len(nums) % 2 != 0:
        raise ValueError("Número impar de coordenadas; faltan valores de lat/lon.")

    # 2) Formamos pares (a, b) según el orden proporcionado
    raw_pairs: List[Tuple[float, float]] = [(nums[i], nums[i+1]) for i in range(0, len(nums), 2)]
    if order not in {"lonlat", "latlon"}:
        raise ValueError("Parámetro 'order' debe ser 'lonlat' o 'latlon'.")

    # 3) Convertimos a (lat, lon) que es lo que espera folium
    if order == "lonlat":
        coords_latlon: List[Tuple[float, float]] = [(lat, lon) for lon, lat in raw_pairs]
    else:  # "latlon"
        coords_latlon = [(lat, lon) for lat, lon in raw_pairs]

    if not coords_latlon:
        raise ValueError("No se encontraron coordenadas en el texto.")

    # 4) Centro del mapa
    center_lat = sum(lat for lat, _ in coords_latlon) / len(coords_latlon)
    center_lon = sum(lon for _, lon in coords_latlon) / len(coords_latlon)

    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom_start, control_scale=True)

    # 5) Marcadores
    for i, (lat, lon) in enumerate(coords_latlon, start=1):
        folium.CircleMarker(location=(lat, lon), radius=4,
                            popup=f"Punto {i}: ({lat:.6f}, {lon:.6f})",
                            tooltip=f"Punto {i}").add_to(m)

    # 6) Polilínea y polígono (si el primer y último punto no coinciden, cerramos el polígono)
    poly = coords_latlon[:]
    if poly[0] != poly[-1]:
        poly.append(poly[0])

    folium.PolyLine(locations=poly, weight=2).add_to(m)
    folium.Polygon(locations=poly, weight=2, fill=True, fill_opacity=0.2).add_to(m)

    if save_path:
        m.save(save_path)
    return m


In [13]:
texto = "120.1972949838888 23.8274792640885,120.2349044157815 23.8163439373747,120.1935496142757 23.68879068070474,120.1401785582926 23.70450947418645,120.1504782409099 23.78821582842221,120.1972949838888 23.8274792640885"
m = map_from_string(texto, order="lonlat", save_path="mi_mapa.html")